## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
# Imports

from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import os
from IPython.display import display, Markdown
from pprint import pprint
import requests
load_dotenv(override=True)

# Constants

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

## We will be making 4 Agents:

1. Search Agent - searches online given a search term using an OpenAI hosted tool
2. Planner Agent - given a query from the user, come up with searches
3. Report Agent - make a report on results
4. Push Agent - send a notification to the user's phone with a summary

## Our First Agent: Search Agent

Given a Search term, search for it on the internet and summarize results.

In [2]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [3]:
message = "What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of May 2025, several AI agent frameworks have gained prominence for their capabilities in developing sophisticated AI applications:

**LangChain**: A leading open-source framework, LangChain simplifies the creation of applications powered by large language models (LLMs). It offers modular components for building complex workflows, integrating memory management, and interacting with external tools and APIs. LangChain supports multiple LLMs, including OpenAI and Hugging Face, making it versatile for various AI tasks. ([curotec.com](https://www.curotec.com/insights/top-ai-agent-frameworks/?utm_source=openai))

**LangGraph**: Building upon LangChain, LangGraph focuses on stateful, multi-agent systems. It utilizes graph-based architectures to manage complex, multi-step tasks, facilitating advanced planning, reflection, and coordination among agents. This framework is particularly useful for applications requiring precise, sequential task execution, such as in healthcare or supply chain management. ([lekha-bhan88.medium.com](https://lekha-bhan88.medium.com/top-5-agentic-ai-frameworks-to-watch-in-2025-9d51b2b652c0?utm_source=openai))

**CrewAI**: CrewAI is designed for orchestrating role-based AI agents, enabling the creation of teams where each agent specializes in specific tasks. It supports dynamic task planning, delegation, and inter-agent communication, making it suitable for complex, multi-step workflows. CrewAI is particularly beneficial for simulating human-like teamwork and collaboration in AI systems. ([chatbase.co](https://www.chatbase.co/blog/ai-agent-frameworks?utm_source=openai))

These frameworks are at the forefront of AI agent development, offering robust tools for building intelligent, autonomous systems across various domains. 

Take a look at the trace

https://platform.openai.com/traces

## Our Second Agent: Planner Agent

Given a query, come up with 5 ideas for web searches that could be run.

Use Structured Outputs as our way to ensure the Agent provides what we need.

In [4]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    output_type=WebSearchPlan,
)

In [5]:

message = "What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='Find up-to-date information about AI Agent frameworks popular in 2025', query='most popular AI agent frameworks 2025'), WebSearchItem(reason='Identify AI Agent frameworks with the highest success and adoption rates in 2025', query='successful AI agent frameworks 2025'), WebSearchItem(reason='Discover which AI Agent platforms or tools developers prefer in 2025', query='top AI agent platforms developers prefer 2025'), WebSearchItem(reason='Look for recent reviews or comparisons of AI Agent frameworks in 2025', query='AI agent framework comparison 2025'), WebSearchItem(reason='Search for expert blog articles or reports on AI agent framework trends in 2025', query='AI agent framework trends 2025')])


## Our Third Agent: Writer Agent

Take the results of internet searches and make a report

In [6]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## Our Fourth Agent: push notification

Just to show how easy it is to make a tool!

I'm using a nifty product called PushOver - to set this up yourself, visit https://pushover.net

In [7]:
@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [8]:
push

FunctionTool(name='push', description='Send a push notification with this brief message', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A446FEA980>, strict_json_schema=True)

In [9]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""


push_agent = Agent(
    name="Push agent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required")
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [10]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and send a push notification

In [11]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    """ Use the push agent to send a notification to the user """
    print("Pushing...")
    result = await Runner.run(push_agent, report.short_summary)
    print("Push sent")
    return report

### Showtime!

In [12]:
query ="What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_push(report)  
    print("Hooray!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Pushing...
Push sent
Hooray!


# Report on Popular and Successful AI Agent Frameworks (May 2025)

## Table of Contents

1. **Introduction**  
2. **Overview of AI Agent Frameworks**  
   - 2.1 Criteria for Selection  
3. **Leading AI Agent Frameworks**  
   - 3.1 LangChain  
   - 3.2 LangGraph  
   - 3.3 CrewAI  
   - 3.4 Microsoft AutoGen  
   - 3.5 Letta  
   - 3.6 Other Notable Frameworks  
4. **Comparative Analysis**  
   - 4.1 Strengths and Weaknesses  
   - 4.2 Use Cases  
5. **Future Trends in AI Agent Frameworks**  
6. **Conclusion**  
7. **References**  

---  

## 1. Introduction  
In recent years, Artificial Intelligence (AI) has seen substantial advancements, particularly in the development of AI agents capable of executing complex tasks autonomously. These agents are powered by sophisticated machine learning algorithms and frameworks that allow for a robust application in several sectors, ranging from customer service to strategic planning. As of May 2025, several frameworks have gained strong traction in this area. This report aims to provide an in-depth analysis of these leading frameworks, highlighting their features, advantages, and applicability in real-world scenarios.

## 2. Overview of AI Agent Frameworks  
### 2.1 Criteria for Selection  
The selection of an AI agent framework depends on various criteria, including:
- **Modularity**: Ability to break complex systems into simpler, manageable components.
- **Integration Capabilities**: Compatibility with existing technologies and systems.
- **Community Support and Resources**: Availability of third-party plugins, tutorials, and community forums.
- **Scalability**: The framework's ability to handle increasing amounts of work or expand its capabilities.
- **Simplicity of Use**: User interface, documentation, and ease of learning for developers.

## 3. Leading AI Agent Frameworks  
### 3.1 LangChain  
LangChain is recognized for its versatility, designed for building applications utilizing Large Language Models (LLMs). Its modular structure simplifies the implementation of workflows, emphasizing context management for tasks ranging from conversational AI to document analysis. Integration with various LLMs, including those from OpenAI and Hugging Face, positions it as a premier choice among developers. However, users report challenges with performance as the system’s complexity increases, alongside a steep learning curve due to the number of features offered ([Curotec](https://www.curotec.com)).

### 3.2 LangGraph  
As an extension of LangChain, LangGraph introduces a graph-based architecture facilitating coordination among multiple agents. With advanced error-handling and visual workflow design, it excels in applications requiring collaborative storytelling or the management of multi-agent systems. Though powerful, developers have noted that it may require a more significant upfront investment to grasp fully, given its complex architecture ([Curotec](https://www.curotec.com)).

### 3.3 CrewAI  
CrewAI focuses on role-based collaboration among AI agents, allowing each to take on specific roles, enhancing the overall teamwork of the system. Its dynamic task planning and inter-agent communication protocols make it suitable for applications requiring human-like decision-making processes. Users appreciate its straightforward implementation, but the newer framework lacks extensive third-party resources and integrations compared to more established options ([Chatbase](https://www.chatbase.co)).

### 3.4 Microsoft AutoGen  
Microsoft AutoGen simplifies the development of multi-agent conversations through its open-source framework. The design supports collaborative problem-solving, making it well-suited for tasks requiring real-time interactions. However, feedback indicates some challenges concerning debugging and limitations in its conversational depth, leading to a slower adoption rate compared to competitors ([Acorn](https://www.acorn.io)).

### 3.5 Letta  
Letta caters to the swift prototyping of modular AI solutions. It integrates seamlessly with various AI libraries, catering especially well to small to medium-scale applications in sectors such as educational technologies. Its high degree of modularity allows for customization and scalability; nonetheless, it can pose challenges during implementation due to its need for a certain level of technical expertise ([Heliverse](https://www.heliverse.com)).

### 3.6 Other Notable Frameworks  
- **OpenAI Agents SDK**: Offers structured tools for building reasoning and planning agents.
- **Smolagents**: Specialized in minimalist, lightweight agents for quick computational tasks, noted for their ease of use but limited in scalability.
- **Agno**: Focuses on multimodal capabilities, praised for its agility but criticized for its complexity in setup ([Langfuse](https://langfuse.com)).

## 4. Comparative Analysis  
### 4.1 Strengths and Weaknesses  
Each framework has specific strengths and weaknesses:
- **LangChain**: Strong community support, flexible, but complex for new users.
- **CrewAI**: Excellent for structured task allocation, but less third-party integration.
- **AutoGen**: Great for dynamic interactions but reported issues during debugging.
- **Letta**: Ideal for rapid prototyping but may overwhelm new users.

### 4.2 Use Cases  
- **LangChain**: Effective for research tools and document analysis.
- **LangGraph**: Best for multi-agent storytelling and strategic planning.
- **CrewAI**: Roles in marketing strategy automation and project management.
- **AutoGen**: Applicable in developing conversational agents and technical support AI.

## 5. Future Trends in AI Agent Frameworks  
The AI landscape is continually evolving. Key trends expected to influence the development of AI agent frameworks include:
- **Increased Focus on Edge Computing**: As organizations seek to process data locally for faster response times, frameworks that support edge solutions will gain prominence.
- **Interoperability Among Frameworks**: The demand for systems capable of collaborating across different frameworks is rising, prompting developers to focus on cross-compatibility in upcoming versions.
- **Enhanced User Experience**: As the complexity of AI systems increases, the need for intuitive, user-friendly frameworks will drive design decisions.

## 6. Conclusion  
As of May 2025, AI agent frameworks are at the forefront of AI development, with several frameworks offering diverse capabilities tailored to various application needs. LangChain, LangGraph, and CrewAI stand out for their unique functionalities and market adoption. When selecting an AI agent framework, developers must consider their specific needs, as each framework provides distinct advantages that cater to different scenarios.

## 7. References  
- Curotec. (2025). Top AI Agent Frameworks. Available at: [Curotec](https://www.curotec.com/insights/top-ai-agent-frameworks/?utm_source=openai)
- Chatbase. (2025). AI Agent Frameworks Analysis. Available at: [Chatbase](https://www.chatbase.co/blog/ai-agent-frameworks?utm_source=openai)
- Acorn. (2025). Understanding Microsoft AutoGen. Available at: [Acorn](https://www.acorn.io/resources/learning-center/ai-agent-frameworks?utm_source=openai)
- Heliverse. (2025). Top AI Agent Frameworks for Developers. Available at: [Heliverse](https://www.heliverse.com/blog/top-5-ai-agent-frameworks-2025?utm_source=openai)
- Langfuse. (2025). Comprehensive Comparison of AI Agent Frameworks. Available at: [Langfuse](https://langfuse.com/blog/2025-03-19-ai-agent-comparison?utm_source=openai)

---

### As always, take a look at the trace

https://platform.openai.com/traces